<a href="https://colab.research.google.com/github/akatwinkle/bab_bab/blob/main/%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import requests
import json
import pandas as pd

def get_year_month():
    year_month = []
    for year in range(2015, 2024+1):
        for month in range(1,12+1):
            year_month.append(f'{year}{month:02d}')
    return year_month

resp = []

for year_month in get_year_month():
    year = year_month[:4]
    month = year_month[4:]

    url = f'http://openapi.seoul.go.kr:8088/4a48554755796f6f36375856727974/json/energyUseDataSummaryInfo/1/100/{year}/{month}'
    api_key = '4a48554755796f6f36375856727974'

    params = {
        'KEY' : api_key,
        'TYPE' : 'json',
        'SERVICE' : 'energyUseDataSummaryInfo',
        'START_INDEX' : '1',
        'END_INDEX' : '1000',
        'YEAR' : year,
        'MON' : month
    }

    try:
        resp = requests.get(url, params=params)
        resp.raise_for_status()

        print("API 호출 성공")

        data = resp.json()['energyUseDataSummaryInfo']["row"]
        data_p = [d for d in data if d['MM_TYPE'] =='개인']
        print(data_p)

    except requests.exceptions.RequestException as e:
        print(f"API 호출 실패: {e}")


API 호출 성공
[{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0'}]
API 호출 성공
[{'YEAR': '2015', 'MON': '02', 'MM_TYPE': '개인', 'CNT': '774620', 'EUS': '189974230', 'EUS1': '193611430', 'EUS2': '200055533', 'ECO2_1': '-6859251.5', 'ECO2_2': '-2908322.636', 'GUS': '56487358', 'GUS1': '59353536', 'GUS2': '66191173', 'GCO2_1': '-6284996.5', 'GCO2_2': '-14078392.16', 'WUS': '12656888.218', 'WUS1': '12713146.172', 'WUS2': '12948410.081', 'WCO2_1': '-173889.9085', 'WCO2_2': '-57731.449622', 'HUS': 

In [57]:
import pandas as pd
import requests
import json

#확인
print(resp.json())
#데이터프레임

df = pd.DataFrame(data_p)

df.info()
df.describe()
df

{'energyUseDataSummaryInfo': {'list_total_count': 7, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2024', 'MON': '12', 'MM_TYPE': '공동주택관리소', 'CNT': '3716', 'EUS': '548480228', 'EUS1': '557799290', 'EUS2': '559693904', 'ECO2_1': '-10266369', 'ECO2_2': '-10266369', 'GUS': '8221238', 'GUS1': '9759209', 'GUS2': '9093999', 'GCO2_1': '-1114991.5', 'GCO2_2': '-2229983', 'WUS': '20262147.7', 'WUS1': '20228097.5', 'WUS2': '20488646.8', 'WCO2_1': '-96224.45', 'WCO2_2': '-288673.35', 'HUS': '0', 'HUS1': '0', 'HUS2': '0', 'HCO2_1': '0', 'HCO2_2': '0', 'REG_DATE': '2025-03-01 04:02:55.0'}, {'YEAR': '2024', 'MON': '12', 'MM_TYPE': '기업', 'CNT': '19451', 'EUS': '633735769', 'EUS1': '641412053', 'EUS2': '669494932', 'ECO2_1': '-21717723.5', 'ECO2_2': '-21717723.5', 'GUS': '30096486', 'GUS1': '45742965', 'GUS2': '34384677', 'GCO2_1': '-9960022.5', 'GCO2_2': '-19920045', 'WUS': '4448777.4', 'WUS1': '4406465.4', 'WUS2': '4396396.4', 'WCO2_1': '47346.5', 'WCO2_2': '142039.5', '

,YEAR,MON,MM_TYPE,CNT,EUS,EUS1,EUS2,ECO2_1,ECO2_2,GUS,...,WUS1,WUS2,WCO2_1,WCO2_2,HUS,HUS1,HUS2,HCO2_1,HCO2_2,REG_DATE
0,2024,12,개인,1250620,134828126,138280636,138879530,-3751957,-3504709,42103424,...,7440605,7625718.9,-206308.35,-573725.128,1513199.277,1598857.184,1781397.386,-176928.008,-376996.699240516,2025-03-01 04:02:55.0


In [52]:
import numpy as np

df = pd.DataFrame(data_p)
df
year_data = df.groupby('YEAR').sum(numeric_only=True).reset_index()
print(year_data)

def get_season(MON):
    if MON in [3, 4, 5]:
        return '봄'
    elif MON in [6, 7, 8]:
        return '여름'
    elif MON in [9, 10, 11]:
        return '가을'
    else:
        return '겨울'

df['year'] = df.index.year
df['season'] = df['MON'].map(get_season)


   YEAR
0  2024


AttributeError: 'RangeIndex' object has no attribute 'year'

<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/tmp/ipython-input-2419853589.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  print(type(['energyUseDataSummaryInfo']["row"]))


TypeError: list indices must be integers or slices, not str